# Notebook 11: Classical vs QML Comparison

**Purpose**: Compare classical ML models with quantum ML models.

**Inputs**:
- `classical_metrics.csv`
- `qml_metrics.csv`

**Outputs**:
- Comparison visualizations → `figures/`

---

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
BASE_DIR = Path('.').resolve().parent
RESULTS_DIR = BASE_DIR / 'results'
FIGURES_DIR = BASE_DIR / 'figures'

# Style
plt.style.use('seaborn-v0_8-whitegrid')
colors = {'classical': '#3498db', 'qml': '#9b59b6'}

In [ ]:
# Load metrics
classical_df = pd.read_csv(RESULTS_DIR / 'classical_metrics.csv')
qml_df = pd.read_csv(RESULTS_DIR / 'qml_metrics.csv')

classical_df['category'] = 'Classical'
qml_df['category'] = 'QML'

combined_df = pd.concat([classical_df, qml_df], ignore_index=True)

print(f"Classical models: {len(classical_df)}")
print(f"QML models: {len(qml_df)}")

In [ ]:
# Display results
print("\nClassical Models:")
print(classical_df[['model', 'f1_score', 'roc_auc', 'train_time']].to_string(index=False))
print("\nQML Models:")
print(qml_df[['model', 'f1_score', 'roc_auc', 'train_time', 'n_qubits']].to_string(index=False))

## Accuracy vs Runtime Trade-offs

In [ ]:
# Performance comparison
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    data = combined_df.sort_values(metric, ascending=True)
    colors_list = [colors['classical'] if c == 'Classical' else colors['qml'] 
                   for c in data['category']]
    
    ax.barh(data['model'], data[metric], color=colors_list)
    ax.set_xlabel(metric.replace('_', ' ').title())
    ax.set_title(f'{metric.replace("_", " ").title()}')
    ax.set_xlim([0, 1])

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors['classical'], label='Classical'),
                   Patch(facecolor=colors['qml'], label='QML')]
axes[-1].legend(handles=legend_elements, loc='center')
axes[-1].axis('off')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_qml_metrics.png', dpi=150)
plt.show()

In [ ]:
# Efficiency scatter plot
fig, ax = plt.subplots(figsize=(12, 8))

for category, color in colors.items():
    data = classical_df if category == 'classical' else qml_df
    
    ax.scatter(data['train_time'], data['f1_score'], 
               c=color, label=category.upper(), s=150, alpha=0.7, edgecolors='black')
    
    for _, row in data.iterrows():
        ax.annotate(row['model'], (row['train_time'], row['f1_score']),
                   fontsize=8, ha='left', va='bottom')

ax.set_xlabel('Training Time (seconds)', fontsize=12)
ax.set_ylabel('F1 Score', fontsize=12)
ax.set_title('Accuracy vs Runtime Trade-off: Classical vs QML', fontsize=14)
ax.legend(fontsize=10)
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_qml_tradeoff.png', dpi=150)
plt.show()

In [ ]:
# Efficiency score: F1 / log(train_time + 1)
combined_df['efficiency'] = combined_df['f1_score'] / np.log1p(combined_df['train_time'])

fig, ax = plt.subplots(figsize=(12, 6))
data = combined_df.sort_values('efficiency', ascending=True)
colors_list = [colors['classical'] if c == 'Classical' else colors['qml'] 
               for c in data['category']]

ax.barh(data['model'], data['efficiency'], color=colors_list)
ax.set_xlabel('Efficiency Score (F1 / log(time))')
ax.set_title('Model Efficiency: Classical vs QML')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_qml_efficiency.png', dpi=150)
plt.show()

In [ ]:
# Summary
print("\n" + "="*60)
print("CLASSICAL VS QML SUMMARY")
print("="*60)

for metric in ['f1_score', 'roc_auc', 'train_time']:
    classical_mean = classical_df[metric].mean()
    qml_mean = qml_df[metric].mean()
    
    print(f"\n{metric.replace('_', ' ').title()}:")
    print(f"  Classical: {classical_mean:.4f}")
    print(f"  QML: {qml_mean:.4f}")
    
    if metric != 'train_time':
        winner = 'Classical' if classical_mean > qml_mean else 'QML'
    else:
        winner = 'Classical' if classical_mean < qml_mean else 'QML'
    print(f"  Winner: {winner}")

print("\n" + "-"*60)
print("KEY INSIGHT: Classical models are typically faster and competitive")
print("on small datasets. QML shows potential but requires more qubits")
print("and circuit depth for complex patterns.")
print("\n✅ Notebook 11 Complete!")